# An end-to-end workflow to efficiently compress and deploy DNN classifiers on SoC/FPGA.

<small>

### DNN Compression Framework

This framework supports flexible compression strategies for student-teacher models, including:

- **Quantization** (QKeras)
- **Pruning** (TF-MOT)
- **Knowledge Distillation (KD)**  
- **LOWRANK**: Replaces full Conv layers with low-rank factorized alternatives.
    - Useful for reducing FLOPs and model size, especially for FPGA targets.
    - Acts as an alternative student model (under `student_2d_lowrank.py`).
- Hyperparameter tuning (KerasTuner)

---

#### 1. Dataset Selection

In `config.py`:

```python
# src/config/config.py
class DatasetConfig:
    D_SIGNAL = 1  # 1D = 1, 2D = 2, CIFAR-10 (or other state-of-the-art (SOTA) dataset) = 3
```

---

#### 2. Model Architecture Reference

This table shows where to find and modify models for different use cases.


| Purpose     | Input Type | File Path                                         | Function Name                     | Description                       |
|-------------|------------|--------------------------------------------------|----------------------------------|-----------------------------------|
| Teacher     | 1D Signal  | src/topology/teacher/teacher_1d.py               | modelTeacherTopology_1D          | Reference dense model             |
| Teacher     | 2D Image   | src/topology/teacher/teacher_2d.py               | modelTeacherTopology_2D          | CNN teacher model for 2D          |
| Teacher     | 2D Image (SOTA)  | src/topology/teacher/teacher_2d_sota.py          | modelTeacherTopology_2D_SOTA     | Deeper model for SOTA dataset     |
| HPO Teacher | 1D Signal  | src/topology/teacher_hpo/teacher_hpo_1d.py       | modelTeacherHPO_1D               | Teacher model for 1D HPO          |
| HPO Teacher | 2D Image   | src/topology/teacher_hpo/teacher_hpo_2d.py       | modelTeacherHPO_2D               | Teacher model for 2D HPO          |
| HPO Teacher | 2D Image (SOTA)   | src/topology/teacher_hpo/teacher_hpo_2d_sota.py  | modelTeacherHPO_2D_SOTA          | Teacher model for SOTA dataset HPO|
| Student     | 1D Signal  | src/topology/student/student_1d.py               | modelKDQP_1D                     | 1D QKeras student model           |
| Student     | 2D Image   | src/topology/student/student_2d.py               | modelStudent2D                   | 2D QKeras student model           |
| Student     | 2D Image (SOTA) | src/topology/student/student_2d_sota.py          | modelStudent2D_SOTA              | 2D QKeras student model           |
| HPO Model   | 1D         | src/topology/student_hpo/student_hpo_1d.py       | modelStudent1D                   | Used for HPO                      |
| HPO Model   | 2D Image   | src/topology/student_hpo/student_hpo_2d.py       | build_model_hpo_student_2d       | Used for HPO                      |
| HPO Model   | 2D Image (SOTA)  | src/topology/student_hpo/student_hpo_2d_sota.py  | build_model_student_hpo_2D_sota  | Used for HPO                      |

SOTA: state-of-the-art

---

### Student Model Selection

You can choose the student model based on the signal type in `student_utils.py`:

```python
MODEL_MAP = {
    1: (modelStudent_1D, (DatasetConfig.SAMPLES,), DatasetConfig.nLabels_1D),
    2: (modelStudent2D, (80, 80, 3), DatasetConfig.nLabels_2D),
    3: (modelStudent2D_SOTA, (32, 32, 3), DatasetConfig.nLabels_2D),
    4: (modelStudentLowRank2D, (32, 32, 3), DatasetConfig.nLabels_2D)  # <-- new key
}
```

--- 

#### 3. Choose Compression Strategy

In `CompressionConfig.py`:

```python
# src/config/CompressionConfig.py
from enum import Enum

class CompressionMode(Enum):
    BASELINE = "baseline"
    QUANTIZATION = "quant"
    PRUNING = "prune"
    KD = "kd"
    Q_KD = "q_kd"
    Q_PRUNING = "q_pruning"
    LOWRANK = "lowrank"
    Q_KD_PRUNING = "q_kd_pruning"

SELECTED_COMPRESSION = CompressionMode.Q_KD_PRUNING
```



---


#### 4. Guidelines

<ul>
  <li>Modify the <strong>student</strong> model for most experiments (pruning, quant, etc.)</li>
  <li>The <strong>teacher</strong> model is only required for <code>KD</code>, <code>Q_KD</code>, and <code>Q_KD_PRUNING</code></li>
  <li>You can switch between model types and input sizes using <code>DatasetConfig.D_SIGNAL</code></li>
  <li>The HPO models must match the selected input and compression mode</li>
</ul>

--- 

#### Run the pipeline

```python
from src.pipeline import startDNNTrainingAndCompression

startDNNTrainingAndCompression()
```

Or customize compression manually:

```python
startDNNTrainingAndCompression(
    use_kd=True,
    use_quant=True,
    use_prune=False
)
```

---

#### Notes

- Teacher model is required for KD-based training.
- Hyperparameter optimization uses Bayesian Optimization via Keras Tuner.
- Model quantization is supported through QKeras.
- Pruning is handled with TensorFlow Model Optimization Toolkit.

---

#### Tips

- For FPGA deployment, use `Q_KD_PRUNING`.


</small>

In [ ]:
import sys
sys.path.append("src")

# from src.config.config import dataset_config, teacher_config, student_config

from src.compressionStart import *

In [2]:
# Default 
startDNNTrainingAndCompression()

KeyboardInterrupt: 